# FOMO-AD Model For Anomaly Detection

# Summary
- Introduction
- Before running the notebook

---

- Config
- Login
- Get your project data

---

- Upload the dataset
- Create an impulse
- Generate features

---

- Train the model
    - Option 1:  Default training configuration
    - Option 2:  Find the best model and configuration from your data

---

- Export your model
- Run the Inference on your exported model

## Introduction

This notebook demonstrate the power of [Edge Impulse](https://edgeimpulse.com/) and it's API to automatically create, train and export a model for anomaly detection in just a few minutes.

Set the configuration, run all the blocks and see your model being trained and exported live from a click of a button.

It will create an anomaly detection model for cookies.

The notebook configuration work for one project at a time.  
Here are the projects for each dataset:
- Cookies 1: https://studio.edgeimpulse.com/public/377841/latest
- Cookies 2: https://studio.edgeimpulse.com/public/371766/latest
- Cookies 3: https://studio.edgeimpulse.com/public/376268/latest




## Before running the notebook

Let's prepare the environement before running the notebook.  

1. Do you have the datasets ? Download it [here](https://drive.google.com/file/d/19VM3RtzVFyDZ4s0HKJ8eVMnijAKEJdRJ/view?usp=drive_link) and extract it in the `notebooks` folder

2. Run the following command in the terminal from the **`ai`** folder.

```sh
conda create -p envs/3_fomoad python=3.11.7 -y
conda activate envs/3_fomoad

pip install -r requirements/3_fomoad.txt
```

## Config

In [2]:
import edgeimpulse_api as ei_api  # Importing Edge Impulse API for managing ML projects  
import os  # Importing OS library for operating system dependent functionality  
from dotenv import load_dotenv  # Import dotenv to load environment variables from a .env file  
import random  # Import random module to generate random numbers  
import requests  # Import requests to make HTTP requests  
import json  # Import JSON to parse JSON data  
import time  # Import time module for time-related tasks  
load_dotenv()  # Load environment variables from .env file  
  
# Configuration dictionary for the project settings  
cfg = {  
    "project_name": "FOMO-AD-cookies-1",  # Name of the project  
    "dataset_path": "datasets/cookies_1",  # Path to dataset directory  
    "project_api_key": os.getenv("EDGE_IMPULSE_API_KEY_FOMO_AD_COOKIES_1"),  # Project API key loaded from environment  
    "project_api_key_dev": os.getenv("EDGE_IMPULSE_API_KEY_FOMO_AD_COOKIES_1_DEV"),  # Development API key loaded from environment  
    "output_path": "../output/fomoad/cookies_1",  # Output directory for storing results  
  
    # Optional, will run a pipeline to find the best model instead of using the default one
    # There's no code to wait for the pipeline to finish, so make sure to run the export block manually once it's done.
    "organization_id": 105154,  # ID of the organization in Edge Impulse  
    "organization_api_key": os.getenv("EDGE_IMPULSE_API_KEY_ORGANISATION"),  # Organization API key loaded from environment  
    "pipeline_id": 3923  # ID of the pipeline to be used to train a model, 
}

## Login

In [3]:
def login():  
    """  
    Log in to the Edge Impulse Studio API and retrieve an authentication token.  
  
    Returns:  
        str: The authentication token used for subsequent API calls.  
    """
    
    # Define the URL to which the POST request will be sent.  
    url = "https://studio.edgeimpulse.com/v1/api-login"  
  
    # Construct payload with environment variables for username and password.  
    payload = json.dumps({  
        "username": os.getenv("EDGE_IMPULSE_USERNAME"),  
        "password": os.getenv("EDGE_IMPULSE_PASSWORD")  
    })  
      
    # Define headers for the POST request.  
    headers = {  
        'content-type': "application/json"  
    }  
      
    # Perform a POST request to log in and capture the response.  
    response = requests.post(url, data=payload, headers=headers)  
    res = response.json()  
      
    # Confirm a successful login in the console.  
    print("Logged in!")  
  
    # Return the API token from the response data.  
    return res["token"]  

token = login()

Logged in!


## Get your project data

In [4]:
def get_active_projects():  
    """Retrieve active projects from the Edge Impulse API.  
  
    Returns:  
        list: A list of active project dictionaries.  
    """  
    response = requests.get(  
        "https://studio.edgeimpulse.com/v1/api/projects",  
        headers={"x-jwt-token": token}  
        )  
    res = response.json()  
    return res["projects"]  
    
projects = get_active_projects()

In [5]:
# Get specific project information based on project name from configuration  
project = next((project for project in projects if project['name'] == cfg["project_name"]), None)

print("ID:", project["id"])
print("Name:", project["name"])
print("Description:", project["description"])

ID: 377841
Name: FOMO-AD-cookies-1
Description: Anomaly Detection model using FOMO AD on cookies dataset type 1


## Upload the dataset

In [6]:
# You can run this multiple time without affecting existing files.

def upload_data(category, files, label):
    """
    Upload data to a specific project category via the Edge Impulse API.  
  
    Args:  
        category (str): The category of data (e.g., 'training', 'testing').  
        files (list): List of file paths to upload.  
        label (str): Label to assign to the uploaded data.  
    """  
    res = requests.post(
        f"https://ingestion.edgeimpulse.com/api/{category}/files", 
        headers={"x-api-key": cfg["project_api_key"], "x-label": label, "x-disallow-duplicates": "1"},
        # Creating the data payload for the request.
        files=(('data', (os.path.basename(i), open(
            i, 'rb'), 'image/jpg')) for i in files)
        )
    
    if (res.status_code == 200):
        print('Uploaded file(s) to Edge Impulse\n', res.status_code, res.content)
    else:
        print('Failed to upload file(s) to Edge Impulse\n',
          res.status_code, res.content)

# Define dataset paths  
dataset_path_good=f"{cfg['dataset_path']}/no_anomaly"
dataset_path_defect_1=f"{cfg['dataset_path']}/anomaly_lvl_1"
dataset_path_defect_2=f"{cfg['dataset_path']}/anomaly_lvl_2"
dataset_path_defect_3=f"{cfg['dataset_path']}/anomaly_lvl_3"

# List all jpg files for good cookies  
all_good_cookies=[f"{dataset_path_good}/{f}" for f in os.listdir(dataset_path_good) if f.endswith('.jpg')]
random.seed(42)
random.shuffle(all_good_cookies)

# Split good cookies data into training and testing sets  
train_good_cookies = all_good_cookies[:-20]
test_good_cookies = all_good_cookies[-20:]

print("train_good_cookies len", len(train_good_cookies))
print("test_good_cookies len", len(test_good_cookies))
# Upload the good cookies dataset  

print("Uploading good cookies training")
upload_data("training", train_good_cookies, "no anomaly") # "no anomaly" and "anomaly" is important!
print("Uploading good cookies testing")
upload_data("testing", test_good_cookies, "no anomaly")

# List all jpg files for defective cookies across different levels  
all_defect_cookies_lvl_1=[f"{dataset_path_defect_1}/{f}" for f in os.listdir(dataset_path_defect_1) if f.endswith('.jpg')]
all_defect_cookies_lvl_2=[f"{dataset_path_defect_2}/{f}" for f in os.listdir(dataset_path_defect_2) if f.endswith('.jpg')]
all_defect_cookies_lvl_3=[f"{dataset_path_defect_3}/{f}" for f in os.listdir(dataset_path_defect_3) if f.endswith('.jpg')]
all_defect_cookies = all_defect_cookies_lvl_1 + all_defect_cookies_lvl_2 + all_defect_cookies_lvl_3  

print("all_defect_cookies", len(all_defect_cookies))

# Upload the defective cookies dataset  
print("Uploading defect cookies testing")
upload_data("testing", all_defect_cookies, "anomaly")
print("Done")

train_good_cookies len 80
test_good_cookies len 20
Uploading good cookies training
Uploaded file(s) to Edge Impulse
 200 b'{"success":true,"files":[{"success":false,"error":"An item with this hash already exists (ids: 933288110)"},{"success":false,"error":"An item with this hash already exists (ids: 933288122)"},{"success":false,"error":"An item with this hash already exists (ids: 933288130)"},{"success":false,"error":"An item with this hash already exists (ids: 933288152)"},{"success":false,"error":"An item with this hash already exists (ids: 933288156)"},{"success":false,"error":"An item with this hash already exists (ids: 933288158)"},{"success":false,"error":"An item with this hash already exists (ids: 933288161)"},{"success":false,"error":"An item with this hash already exists (ids: 933288162)"},{"success":false,"error":"An item with this hash already exists (ids: 933288163)"},{"success":false,"error":"An item with this hash already exists (ids: 933288164)"},{"success":false,"erro

## Create an impulse

In [6]:
def get_impulse(projectId):  
    """    
    Fetches the impulse configuration for a given project ID.  
        
    Args:    
        projectId: str. ID of the project from which to fetch the impulse configuration.  
        
    Returns:    
        dict: The impulse configuration in JSON format.  
    """    
    response = requests.get(f"https://studio.edgeimpulse.com/v1/api/{projectId}/impulse",  
                            headers={"x-jwt-token": token})  
    res = response.json()  
    return res 

# get_impulse(project_fomoad["id"])
get_impulse(project["id"])

{'success': True,
 'impulse': {'inputBlocks': [{'db': False,
    'id': 11,
    'name': 'Image data',
    'type': 'image',
    'title': 'Image data',
    'enabled': True,
    'createdBy': 'clone',
    'cropAnchor': 'middle-center',
    'imageWidth': 160,
    'resizeMode': 'squash',
    'description': 'EON Tuner Primary',
    'imageHeight': 160,
    'resizeMethod': 'lanczos3',
    'labelingMethod': 'single_label',
    'tunerTemplateId': -1,
    'createdAt': '2024-04-16T08:16:51.849Z',
    'mutated': False,
    'clonedFromBlockId': 4516308,
    'primaryVersion': True}],
  'dspBlocks': [{'db': False,
    'id': 12,
    'axes': ['image'],
    'name': 'Image',
    'type': 'image',
    'input': 11,
    'title': 'Image',
    'enabled': True,
    'autotune': False,
    'createdBy': 'clone',
    'description': 'EON Tuner Primary',
    'tunerTemplateId': -1,
    'implementationVersion': 1,
    'createdAt': '2024-04-16T08:16:51.849Z',
    'mutated': False,
    'clonedFromBlockId': 4516309,
    'pri

In [8]:
# DEFAULT LEARNING BLOCK
# I got it from creating one manually then reading the API output from the browser's console.
# It might not be up to date with the current one. Last update: April 2024.
default_impulse_payload = {
    "inputBlocks": [
      {
        "db": False,
        "id": 11,
        "name": "Image data",
        "type": "image",
        "title": "Image data",
        "enabled": True,
        "createdBy": "clone",
        "cropAnchor": "middle-center",
        "imageWidth": 160,
        "resizeMode": "squash",
        "description": "EON Tuner Primary",
        "imageHeight": 160,
        "resizeMethod": "lanczos3",
        "labelingMethod": "single_label",
        # "primaryVersion": True,
        "tunerTemplateId": -1,
        "createdAt": "2024-04-16T08:16:51.849Z",
        "mutated": False,
        "clonedFromBlockId": 4516308,
        # "tunerPrimary": True
      }
    ],
    "dspBlocks": [
      {
        "db": False,
        "id": 12,
        "axes": [
          "image"
        ],
        "name": "Image",
        "type": "image",
        "input": 11,
        "title": "Image",
        "enabled": True,
        "autotune": False,
        "createdBy": "clone",
        "description": "EON Tuner Primary",
        # "primaryVersion": True,
        "tunerTemplateId": -1,
        "implementationVersion": 1,
        "createdAt": "2024-04-16T08:16:51.849Z",
        "mutated": False,
        "clonedFromBlockId": 4516309,
        # "tunerPrimary": True
      }
    ],
    "learnBlocks": [
      {
        "id": 15,
        "type": "keras-visual-anomaly",
        "name": "FOMO-AD",
        "dsp": [
          12
        ],
        "title": "FOMO-AD (Images)",
        "primaryVersion": True,
        "createdBy": "createImpulse",
        "createdAt": "2024-04-16T09:04:22.040Z"
      }
    ]
}

In [9]:
def delete_impulse(projectId):  
    """    
    Deletes the impulse configuration for a given project ID.  
        
    Args:    
        projectId: str. ID of the project for which to delete the impulse configuration.  
        
    Returns:    
        dict: The result of the deletion in JSON format.  
    """    
    response = requests.delete(f"https://studio.edgeimpulse.com/v1/api/{projectId}/impulse",  
                               headers={"x-jwt-token": token})  
    res = response.json()  
    return res  

def create_impulse(projectId, payload):  
    """    
    Creates or updates the impulse configuration for a project based on the provided payload.  
        
    Args:    
        projectId: str. ID of the project for which to create or update the impulse configuration.  
        payload: dict. Configuration details necessary to create or update the impulse.  
        
    Returns:    
        dict: The result of the impulse creation or update in JSON format.  
    """    
    delete_impulse(projectId)  
    response = requests.post(f"https://studio.edgeimpulse.com/v1/api/{projectId}/impulse",  
                             json=payload, headers={"x-jwt-token": token})  
    res = response.json()  
    return res  

create_impulse(project["id"], default_impulse_payload)

{'success': True}

## Generate features

In [12]:
def get_jobs(projectId):  
    """    
    Fetches all jobs for a given project ID.  
        
    Args:    
        projectId: str. ID of the project from which to fetch all jobs.  
        
    Returns:    
        dict: Information about all jobs in JSON format.  
    """    
    response = requests.get(f"https://studio.edgeimpulse.com/v1/api/{projectId}/jobs/all",  
                            headers={"x-jwt-token": token})  
    res = response.json()  
    return res  

def get_job_status(projectId, jobId):  
    """    
    Fetches the status of a specific job for a given project ID.  
        
    Args:    
        projectId: str. ID of the project.  
        jobId: str. Specific job ID to fetch the status for.  
        
    Returns:    
        dict: Status of the job in JSON format.  
    """  
    response = requests.get(f"https://studio.edgeimpulse.com/v1/api/{projectId}/jobs/{jobId}/status",  
                            headers={"x-jwt-token": token})  
    res = response.json()  
    return res["job"]

def get_job_output(projectId, jobId):  
    """    
    Fetches the output of a specific job for a given project ID.  
        
    Args:    
        projectId: str. ID of the project.  
        jobId: str. Specific job ID to fetch the output for.  
        
    Returns:    
        dict: Output of the job in JSON format.  
    """  
    response = requests.get(f"https://studio.edgeimpulse.com/v1/api/{projectId}/jobs/{jobId}/stdout",  
                            headers={"x-jwt-token": token})  
    res = response.json()  
    return res 

def generate_feature(projectId):  
    """    
    Generates feature data for a project based on specific DSP configuration parameters.  
        
    Args:    
        projectId: str. ID of the project to generate features for.  
        
    Returns:    
        str: ID of the job handling feature generation.  
    """   
    payload = {  
        "dspId": 12,  
        "calculateFeatureImportance": False  
    }  
    response = requests.post(f"https://studio.edgeimpulse.com/v1/api/{projectId}/jobs/generate-features",  
                             json=payload, headers={"x-jwt-token": token})  
    res = response.json()  
    return res["id"]  

In [11]:
job_id = generate_feature(project["id"])
print(job_id)

18483524


In [12]:
get_job_status(project["id"], job_id)

{'id': 18483524,
 'category': 'Generating features (Image)',
 'key': 'dsp-studio-wrapper-12',
 'created': '2024-04-17T15:33:30.865Z',
 'jobNotificationUids': [],
 'additionalInfo': 'Image',
 'computeTime': 0,
 'createdByUser': {'id': 175414,
  'name': 'math-pro',
  'username': 'callmemath-pro'}}

In [13]:
# Wait until the job is done
while True:  
    status = get_job_status(project["id"], job_id)  
    if "finishedSuccessful" in status and status["finishedSuccessful"]:
        print("Feature generated!") 
        break  
    if "finished" in status and status["finished"]:
        print("Generating done not successful", status) 
        break
    
    print("- Generating feature still in progress, waiting 10sec")
    time.sleep(10)

- Generating feature still in progress, waiting 10sec
- Generating feature still in progress, waiting 10sec
- Generating feature still in progress, waiting 10sec
- Generating feature still in progress, waiting 10sec
- Generating feature still in progress, waiting 10sec
Feature generated!


## Train the model

### Option 1: Default training configuration

In [ ]:
def train_model_keras(projectId, learnId):  
    """    
    Trigger a machine learning model training on a Edge Impulse
    using the proper configuration for anomaly training.
        
    Args:    
        projectId: str. Unique identifier for the project.  
        learnId: str. Identifier for the learning block to be trained.  
        
    Returns:    
        str: Unique job identifier for the triggered training.  
    """
    
    # Set up the payload with training parameters (Seen on the API while testing manually on the browser)
    payload = {
        "trainTestSplit": 0.2,
        "customValidationMetadataKey": "",
        "autoClassWeights": False,
        "profileInt8": True,
        "learningRate": 0.01,
        "trainingCycles": 1,
        "visualLayers": [
            {
                "type": "transfer_mobilenetv2_a35"
            }
        ],
        "augmentationPolicyImage": "none",
        "anomalyCapacity": "low",
        "customParameters": {}
    }
    
    # Make an API request to start training  
    response = requests.post(f"https://studio.edgeimpulse.com/v1/api/{projectId}/jobs/train/keras/{learnId}", json=payload, headers={"x-jwt-token": token})
    res = response.json()
    return res["id"]

# If you haven't defined an organisation, run the training
if cfg["organization_id"] == 0:
    
    # Start training and get job ID  
    job_id_train = train_model_keras(project["id"], default_impulse_payload["learnBlocks"][0]["id"])
    
    while True:
        # Monitor the status of the training job  
        status = get_job_status(project["id"], job_id_train)
        
        if "finishedSuccessful" in status and status["finishedSuccessful"]:
            print("Training done") 
            break
        
        if "finished" in status and status["finished"]:
            print("Training done not successful", status) 
            break
        
        print("- Training is still in progress, waiting 10sec...")
        time.sleep(10)
        
else:
    print("Skipping training, will use transformation blocks instead")
    

Skipping training, will use transformation blocks instead


### Option 2: Find the best model and configuration from your data

In [65]:
def run_pipeline():
    """  
    Initiates a run of the configured data pipeline
      
    Returns:  
        dict: The JSON response from the API.  
    """  
    print(f"https://studio.edgeimpulse.com/v1/api/organizations/{cfg['organization_id']}/pipelines/{cfg['pipeline_id']}/run")
    response = requests.post(f"https://studio.edgeimpulse.com/v1/api/organizations/{cfg['organization_id']}/pipelines/{cfg['pipeline_id']}/run", headers={"x-api-key": cfg["organization_api_key"]})
    return response.json()

# Only run if an organization is defined
if cfg["organization_id"]:
    pass
else:
    pipeline = run_pipeline()



https://studio.edgeimpulse.com/v1/api/organizations/105154/pipelines/3928/run


{'success': True,
 'pipelineRun': {'id': 429023,
  'created': '2024-04-17T17:09:02.519Z',
  'steps': [{'name': 'Import data using "Find best Visual AD model" - Import data using Find best Visual AD model - 2024-04-17T17:09:02.448Z',
    'transformationJob': {'id': 576931,
     'organizationId': 105154,
     'name': 'Import data using "Find best Visual AD model" - Import data using Find best Visual AD model - 2024-04-17T17:09:02.448Z',
     'uploadJobStatus': 'waiting',
     'uploadType': 'project',
     'projectId': 0,
     'projectName': '',
     'transformationBlockId': 5671,
     'transformationBlockName': 'Find best Visual AD model',
     'category': 'training',
     'created': '2024-04-17T17:09:02.459Z',
     'totalDownloadFileCount': 0,
     'totalDownloadFileSize': 0,
     'totalDownloadFileSizeString': '0 Bytes',
     'totalUploadFileCount': 0,
     'transformationSummary': {'startedCount': 0,
      'finishedCount': 0,
      'succeededCount': 0,
      'totalFileCount': 0,
     

## Export your model

In [9]:
def get_deployment_targets(projectId):  
    """  
    Retrieves deployment targets available for a given project.  
      
    Args:  
        projectId (str): The project ID  
      
    Returns:  
        list: A list of deployment targets  
    """  

    response = requests.get(f"https://studio.edgeimpulse.com/v1/api/{projectId}/deployment/targets", headers={"x-jwt-token": token})
    res = response.json()
    return res["targets"]

def build_model(projectId, eim_type="runner-mac-x86_64"):  
    """  
    Requests the building of an on-device model for a specified project.  
      
    Args:  
        projectId (str): The project ID.  
        eim_type (str): Type of the executable inference model.  
      
    Returns:  
        str: The job ID for the model building task.  
    """  
    payload = {
       "engine": "tflite"
    }
    response = requests.post(f"https://studio.edgeimpulse.com/v1/api/{projectId}/jobs/build-ondevice-model?type={eim_type}", json=payload, headers={"x-jwt-token": token})
    res = response.json()
    return res["id"]

def download_model_built(projectId, eim_type="runner-mac-x86_64"):  
    """  
    Retrieves a built model for a specified project and model type.  
      
    Args:  
        projectId (str): The project ID.  
        eim_type (str): Type of the executable inference model.  
      
    Returns:  
        Response: A response object containing the binary model data.  
    """ 
    payload = {
       "engine": "tflite"
    }
    response = requests.get(f"https://studio.edgeimpulse.com/v1/api/{projectId}/deployment/download?type={eim_type}", headers={"x-jwt-token": token})
    return response

def build_and_download_model(projectId, extension="eim",eim_type="runner-mac-x86_64"):  
    """  
    Orchestrates the building and downloading of a model for a specified project.  
      
    Args:  
        projectId (str): The project ID.  
        extension (str): File extension for the downloaded model.  
        eim_type (str): Type of the executable inference model.  
    """ 
    job_id_build = build_model(projectId)
    while True:
        status = get_job_status(projectId, job_id_build)  
        if "finishedSuccessful" in status and status["finishedSuccessful"]:
            print("Building done") 
            break  
        if "finished" in status and status["finished"]:
            print("Building done not successful", status) 
            break  
        
        print("- Building is still in progress, waiting 10sec...")
        time.sleep(10)
        
    data = download_model_built(projectId)
    os.makedirs(cfg["output_path"], exist_ok=True)
    with open(f'{cfg["output_path"]}/fomoad_{eim_type}.{extension}', 'wb') as f:
        f.write(data.content)


In [13]:
build_and_download_model(project["id"])
build_and_download_model(project["id"], extension="wasm", eim_type="wasm")

- Building is still in progress, waiting 10sec...
- Building is still in progress, waiting 10sec...
- Building is still in progress, waiting 10sec...
- Building is still in progress, waiting 10sec...
Building done
- Building is still in progress, waiting 10sec...
- Building is still in progress, waiting 10sec...
- Building is still in progress, waiting 10sec...
- Building is still in progress, waiting 10sec...
Building done


In [14]:
# For the macos model only, modify this path if needed
!chmod +x "../output/fomoad/cookies_1/fomoad_runner-mac-x86_64.eim"

## Run the Inference on your exported model

In [19]:
from edge_impulse_linux.image import ImageImpulseRunner
import cv2  

modelfile = cfg["output_path"]

def ei_inference(img_path):
    with ImageImpulseRunner(modelfile) as runner:
        model_info = runner.init()
        # print("model_info", model_info)
        
        # Load the image directly from the disk  
        original_image = cv2.imread(img_path, cv2.IMREAD_COLOR)  
        # Convert the image from BGR to RGB (since OpenCV loads images in BGR format)  
        img = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  
        
        features, cropped = runner.get_features_from_image(img)
        # print("FEATURES", len(features), len(cropped))
        
        # print("GO")
        res = runner.classify(features)
        print(res["result"])



all_defect_cookies_lvl_1=[f"{dataset_path_defect_1}/{f}" for f in os.listdir(dataset_path_defect_1) if f.endswith('.jpg')]
all_defect_cookies_lvl_2=[f"{dataset_path_defect_2}/{f}" for f in os.listdir(dataset_path_defect_2) if f.endswith('.jpg')]
all_defect_cookies_lvl_3=[f"{dataset_path_defect_3}/{f}" for f in os.listdir(dataset_path_defect_3) if f.endswith('.jpg')]
all_defect_cookies = all_defect_cookies_lvl_1 + all_defect_cookies_lvl_2 + all_defect_cookies_lvl_3  

print("Original is good")
ei_inference(test_good_cookies[0])
print("\nOriginal is anomaly lvl 1")
ei_inference(all_defect_cookies_lvl_1[0])
print("\nOriginal is anomaly lvl 2")
ei_inference(all_defect_cookies_lvl_2[0])
ei_inference( "datasets/cookies_2/anomaly_lvl_2/20240417_142025.jpg")
print("\nOriginal is anomaly lvl 3")
ei_inference(all_defect_cookies_lvl_3[0])

Original is good
{'anomaly': 0.0, 'visual_anomaly_grid': [], 'visual_anomaly_max': 2.5091969966888428, 'visual_anomaly_mean': 0.8197279572486877}

Original is anomaly lvl 1
{'anomaly': 0.0, 'visual_anomaly_grid': [{'height': 16, 'label': 'anomaly', 'value': 6.74155855178833, 'width': 16, 'x': 151, 'y': 101}, {'height': 16, 'label': 'anomaly', 'value': 4.752529621124268, 'width': 16, 'x': 117, 'y': 117}, {'height': 16, 'label': 'anomaly', 'value': 12.333181381225586, 'width': 16, 'x': 151, 'y': 117}, {'height': 16, 'label': 'anomaly', 'value': 4.555690288543701, 'width': 16, 'x': 168, 'y': 117}, {'height': 16, 'label': 'anomaly', 'value': 7.292279243469238, 'width': 16, 'x': 185, 'y': 117}, {'height': 16, 'label': 'anomaly', 'value': 7.878374099731445, 'width': 16, 'x': 202, 'y': 117}, {'height': 16, 'label': 'anomaly', 'value': 9.117081642150879, 'width': 16, 'x': 151, 'y': 134}, {'height': 16, 'label': 'anomaly', 'value': 7.489037990570068, 'width': 16, 'x': 168, 'y': 134}, {'height':